In [74]:
from experiment import ResultsViewer
from IPython.display import Image
import torch
from scipy.io import savemat
import os

In [75]:
res = ResultsViewer("experiments/experiment_1_fixed_term_2")

def lookup(results, *filters):
  r = []
  for res in results:
    skip = False
    for f in filters:
      if not f(res):
        skip = True
        break
    if not skip:
      r.append(res)
  return r

def save_to_matlab(res):
  os.makedirs("matlab_models", exist_ok=True)
  W = []
  V = []
  skip = []
  out2 = {}
  for i in range(res['model']['layers']):
    key = 'blocks.{}.{}.weight'
    w = sd[key.format(i, 'W')].numpy()
    v = sd[key.format(i, 'V')].numpy()  
    s = sd[key.format(i, 'skip_l')].numpy()  
    W.append(w)
    V.append(v)
    skip.append(s)
    out2[f'W_{i}'] = w
    out2[f'V_{i}'] = v
    out2[f'skip_{i}'] = s
  out = {"W": W, "V": V, "skip": skip}
  fname = f"D={res['data']['D']}_Term={res['model']['regularization_method']}_Lam={res['model']['regularization_lambda']}.mat"
  savemat("matlab_models/" + fname, out2)

In [82]:
r = lookup(
  res,
  lambda x: x['model']['regularization_method'] == 1,
  lambda x: x['model']['regularization_lambda'] == 0.001,
  lambda x: x['model']['start_regularization'] == 10000,
)
len(r)

1

In [83]:
save_to_matlab(r[0])

In [84]:
loadmat("matlab_models/D=4_Term=2_Lam=0.001.mat")

{'__header__': b'MATLAB 5.0 MAT-file Platform: posix, Created on: Tue May 25 12:09:43 2021',
 '__version__': '1.0',
 '__globals__': [],
 'W_0': array([[-2.3155e-41, -2.0767e-41,  1.2612e-42,  5.7717e-41],
        [ 1.0005e-42,  3.4052e-43,  1.8049e-41, -1.0194e-41],
        [ 1.1408e-41,  3.2790e-42, -2.2753e-41,  5.4269e-41],
        ...,
        [ 1.9489e-41,  9.8679e-42, -3.9032e-41,  4.3859e-41],
        [ 2.2156e-41, -1.6888e-41,  1.0314e-41, -1.5620e-41],
        [-1.4366e-41, -5.8560e-42, -6.3578e-41, -2.9026e-41]],
       dtype=float32),
 'V_0': array([[ 1.3854418e-09, -5.6019709e-41,  1.8314172e-09, ...,
          7.1450972e-07, -3.3939449e-42,  4.3660256e-41],
        [-4.8577413e-41,  7.6987338e-42, -3.2618024e-41, ...,
          1.1126310e-41,  9.9828503e-42,  2.3597866e-42],
        [ 2.8691586e-41, -8.0104354e-11, -5.8286970e-10, ...,
         -6.7950673e-11,  1.8746571e-41, -1.5405839e-13],
        ...,
        [ 5.4077509e-41, -6.3173337e-41,  1.7251385e-41, ...,
      